# 🎯 Multi-Layer Analytics - Real Data Demo

Demo training dan inference menggunakan **data real** dari dataset timnas.

**Layers:**
- 😊 Emotion (multi-label): marah, kecewa, sedih, senang, bangga, takut
- 🎯 Aspect (multi-label): manajemen, pelatih, pemain, strategi, wasit, PSSI, federasi, fanbase
- ⚠️ Toxicity (binary): toxic, non-toxic
- 🔀 Stance (3-class): pro, kontra, tidak_jelas
- 💬 Intent (6-class): pertanyaan, komplain, saran, ajakan, humor, informasi

## 1. Setup & Import

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../src'))

import pandas as pd
import numpy as np
import yaml
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Import layered classifiers
from layered_classifier import (
    EmotionClassifier,
    AspectClassifier,
    ToxicityClassifier,
    StanceClassifier,
    IntentClassifier
)

# Import utilities
from utils.layered_utils import (
    create_layered_output_dataframe,
    create_comprehensive_report,
    plot_label_distribution,
    multilabel_to_string,
    string_to_multilabel
)

print("✅ All imports successful!")

## 2. Load Configuration

In [ ]:
# Load config
config_path = Path('../config.yaml')
with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

layered_config = config['layered_analytics']
print("📋 Configuration loaded:")
print(f"  - Emotion labels: {layered_config['emotion_labels']}")
print(f"  - Aspect labels: {layered_config['aspect_labels']}")
print(f"  - Stance labels: {layered_config['stance_labels']}")
print(f"  - Intent labels: {layered_config['intent_labels']}")
print(f"  - Low confidence threshold: {layered_config['low_confidence_threshold']}")
print(f"  - Toxicity threshold: {layered_config['toxicity_threshold']}")

## 3. Load Real Data

In [ ]:
# Load processed real data
data_path = Path('../data/processed/final_processed_dataset_real.csv')
df = pd.read_csv(data_path)

print(f"📊 Data loaded: {len(df)} comments")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nSentiment distribution:")
print(df['sentiment_auto'].value_counts())

# Preview data
print("\n📝 Sample comments:")
df.head()

## 4. Generate Dummy Labels untuk Demo

**Note:** Untuk production, labels harus di-label manual mengikuti `data/labelled/LABELING_GUIDE.md`.

Untuk demo ini, kita generate dummy labels based on sentiment dan keywords.

In [ ]:
def generate_dummy_labels(df):
    """
    Generate dummy labels untuk demo purposes.
    UNTUK PRODUCTION: Label manual menggunakan LABELING_GUIDE.md!
    """
    df = df.copy()
    
    # Emotion labels (multi-label) based on sentiment + keywords
    emotions = []
    for idx, row in df.iterrows():
        text = row['text_cleaned'].lower()
        sent = row['sentiment_auto']
        
        emotion_list = []
        if sent == 'negatif':
            if any(word in text for word in ['kecewa', 'buruk', 'gagal']):
                emotion_list.append('kecewa')
            if any(word in text for word in ['bodoh', 'payah', 'berantakan']):
                emotion_list.append('marah')
            if any(word in text for word in ['sedih', 'menyedihkan']):
                emotion_list.append('sedih')
        elif sent == 'positif':
            if any(word in text for word in ['bangga', 'hebat', 'luar biasa', 'salut']):
                emotion_list.append('bangga')
            if any(word in text for word in ['senang', 'semangat']):
                emotion_list.append('senang')
        
        if not emotion_list:  # default
            emotion_list = ['senang'] if sent == 'positif' else ['kecewa'] if sent == 'negatif' else ['senang']
        
        emotions.append(emotion_list)
    
    df['emotions'] = emotions
    
    # Aspect labels (multi-label) based on keywords
    aspects = []
    for idx, row in df.iterrows():
        text = row['text_cleaned'].lower()
        aspect_list = []
        
        if any(word in text for word in ['manajemen', 'pssi', 'berantakan']):
            aspect_list.append('manajemen')
        if any(word in text for word in ['pelatih', 'shin', 'tae-yong', 'coach']):
            aspect_list.append('pelatih')
        if any(word in text for word in ['pemain', 'timnas', 'anak bangsa']):
            aspect_list.append('pemain')
        if any(word in text for word in ['strategi', 'defensif', 'menyerang']):
            aspect_list.append('strategi')
        if any(word in text for word in ['pssi']):
            aspect_list.append('PSSI')
        
        if not aspect_list:  # default
            aspect_list = ['pemain']
        
        aspects.append(aspect_list)
    
    df['aspects'] = aspects
    
    # Toxicity (binary) - detect offensive language
    toxic_keywords = ['bodoh', 'payah', 'berantakan', 'buruk sekali']
    df['toxicity_label'] = df['text_cleaned'].apply(
        lambda x: 'toxic' if any(word in x.lower() for word in toxic_keywords) else 'non-toxic'
    )
    
    # Stance (3-class) based on sentiment
    stance_map = {
        'positif': 'pro',
        'negatif': 'kontra',
        'netral': 'tidak_jelas'
    }
    df['stance'] = df['sentiment_auto'].map(stance_map)
    
    # Intent (6-class) based on patterns
    intents = []
    for idx, row in df.iterrows():
        text = row['text_cleaned'].lower()
        sent = row['sentiment_auto']
        
        if '?' in text:
            intent = 'pertanyaan'
        elif sent == 'negatif' and any(word in text for word in ['harus', 'kurang', 'buruk']):
            intent = 'komplain'
        elif any(word in text for word in ['evaluasi', 'belajar', 'semangat']):
            intent = 'saran'
        elif any(word in text for word in ['dukung', 'ayo', 'tetap']):
            intent = 'ajakan'
        elif sent == 'positif':
            intent = 'informasi'
        else:
            intent = 'informasi'
        
        intents.append(intent)
    
    df['intent'] = intents
    
    return df

# Generate dummy labels
df_labelled = generate_dummy_labels(df)

print("✅ Dummy labels generated!")
print("\n⚠️ REMINDER: Untuk production, gunakan manual labeling dengan LABELING_GUIDE.md")
print("\n📊 Label distribution:")
print(f"\nEmotion: {df_labelled['emotions'].value_counts().head()}")
print(f"\nAspect: {df_labelled['aspects'].value_counts().head()}")
print(f"\nToxicity:\n{df_labelled['toxicity_label'].value_counts()}")
print(f"\nStance:\n{df_labelled['stance'].value_counts()}")
print(f"\nIntent:\n{df_labelled['intent'].value_counts()}")

# Preview labelled data
df_labelled[['text_cleaned', 'sentiment_auto', 'emotions', 'aspects', 'toxicity_label', 'stance', 'intent']].head()

## 5. Prepare Training Data

Dengan data terbatas (7 samples), kita akan:
- Train pada semua data (no test split karena terlalu sedikit)
- Evaluate pada training set untuk demonstrasi
- **Untuk production:** Butuh minimal 300-500 samples per class

In [ ]:
# Use all data for training (karena dataset kecil)
X_train = df_labelled['text_cleaned'].values
X_test = X_train  # Same data untuk evaluation demo

print(f"📊 Training samples: {len(X_train)}")
print(f"📊 Test samples: {len(X_test)} (same as train for demo)")
print("\n⚠️ Production note: Split 80/20 dengan dataset >= 500 samples")

## 6. Train Layer 1: Emotion Classifier (Multi-Label)

In [ ]:
# Prepare emotion labels
y_train_emotion = df_labelled['emotions'].values
y_test_emotion = y_train_emotion

# Initialize and train
emotion_clf = EmotionClassifier(
    emotion_labels=layered_config['emotion_labels'],
    max_features=5000,
    ngram_range=(1, 2)
)

print("🔄 Training Emotion Classifier...")
emotion_clf.train(X_train, y_train_emotion)

# Evaluate
metrics = emotion_clf.evaluate(X_test, y_test_emotion)
print("\n✅ Emotion Classifier trained!")
print(f"\n📊 Metrics:")
print(f"  - Macro F1: {metrics['macro_f1']:.3f}")
print(f"  - Hamming Loss: {metrics['hamming_loss']:.3f}")
print(f"  - Jaccard Score: {metrics['jaccard_score']:.3f}")

# Save model
model_path = '../models/emotion_classifier.pkl'
emotion_clf.save(model_path)
print(f"\n💾 Model saved: {model_path}")

## 7. Train Layer 2: Aspect Classifier (Multi-Label)

In [ ]:
# Prepare aspect labels
y_train_aspect = df_labelled['aspects'].values
y_test_aspect = y_train_aspect

# Initialize and train
aspect_clf = AspectClassifier(
    aspect_labels=layered_config['aspect_labels'],
    max_features=5000,
    ngram_range=(1, 2)
)

print("🔄 Training Aspect Classifier...")
aspect_clf.train(X_train, y_train_aspect)

# Evaluate
metrics = aspect_clf.evaluate(X_test, y_test_aspect)
print("\n✅ Aspect Classifier trained!")
print(f"\n📊 Metrics:")
print(f"  - Macro F1: {metrics['macro_f1']:.3f}")
print(f"  - Hamming Loss: {metrics['hamming_loss']:.3f}")
print(f"  - Jaccard Score: {metrics['jaccard_score']:.3f}")

# Save model
model_path = '../models/aspect_classifier.pkl'
aspect_clf.save(model_path)
print(f"\n💾 Model saved: {model_path}")

## 8. Train Layer 3: Toxicity Classifier (Binary)

In [ ]:
# Prepare toxicity labels
y_train_toxicity = df_labelled['toxicity_label'].values
y_test_toxicity = y_train_toxicity

# Initialize and train
toxicity_clf = ToxicityClassifier(
    max_features=5000,
    ngram_range=(1, 2)
)

print("🔄 Training Toxicity Classifier...")
toxicity_clf.train(X_train, y_train_toxicity)

# Evaluate
metrics = toxicity_clf.evaluate(X_test, y_test_toxicity)
print("\n✅ Toxicity Classifier trained!")
print(f"\n📊 Metrics:")
print(f"  - Accuracy: {metrics['accuracy']:.3f}")
print(f"  - Precision: {metrics['precision']:.3f}")
print(f"  - Recall: {metrics['recall']:.3f}")
print(f"  - F1: {metrics['f1']:.3f}")

# Save model
model_path = '../models/toxicity_classifier.pkl'
toxicity_clf.save(model_path)
print(f"\n💾 Model saved: {model_path}")

## 9. Train Layer 4: Stance Classifier (3-Class)

In [ ]:
# Prepare stance labels
y_train_stance = df_labelled['stance'].values
y_test_stance = y_train_stance

# Initialize and train
stance_clf = StanceClassifier(
    stance_labels=layered_config['stance_labels'],
    max_features=5000,
    ngram_range=(1, 2)
)

print("🔄 Training Stance Classifier...")
stance_clf.train(X_train, y_train_stance)

# Evaluate
metrics = stance_clf.evaluate(X_test, y_test_stance)
print("\n✅ Stance Classifier trained!")
print(f"\n📊 Metrics:")
print(f"  - Accuracy: {metrics['accuracy']:.3f}")
print(f"  - Macro Precision: {metrics['macro_precision']:.3f}")
print(f"  - Macro Recall: {metrics['macro_recall']:.3f}")
print(f"  - Macro F1: {metrics['macro_f1']:.3f}")

# Save model
model_path = '../models/stance_classifier.pkl'
stance_clf.save(model_path)
print(f"\n💾 Model saved: {model_path}")

## 10. Train Layer 5: Intent Classifier (6-Class)

In [ ]:
# Prepare intent labels
y_train_intent = df_labelled['intent'].values
y_test_intent = y_train_intent

# Initialize and train
intent_clf = IntentClassifier(
    intent_labels=layered_config['intent_labels'],
    max_features=5000,
    ngram_range=(1, 2)
)

print("🔄 Training Intent Classifier...")
intent_clf.train(X_train, y_train_intent)

# Evaluate
metrics = intent_clf.evaluate(X_test, y_test_intent)
print("\n✅ Intent Classifier trained!")
print(f"\n📊 Metrics:")
print(f"  - Accuracy: {metrics['accuracy']:.3f}")
print(f"  - Macro Precision: {metrics['macro_precision']:.3f}")
print(f"  - Macro Recall: {metrics['macro_recall']:.3f}")
print(f"  - Macro F1: {metrics['macro_f1']:.3f}")

# Save model
model_path = '../models/intent_classifier.pkl'
intent_clf.save(model_path)
print(f"\n💾 Model saved: {model_path}")

## 11. Inference: Apply All Layers pada Data Real

In [ ]:
# Get predictions from all layers
texts = df['text_cleaned'].values

print("🔄 Applying all layers...\n")

# Layer 1: Emotion
emotion_preds, emotion_probs = emotion_clf.predict(texts, return_probabilities=True)
print("✅ Emotion predictions done")

# Layer 2: Aspect
aspect_preds, aspect_probs = aspect_clf.predict(texts, return_probabilities=True)
print("✅ Aspect predictions done")

# Layer 3: Toxicity
toxicity_labels, toxicity_scores = toxicity_clf.predict(texts, return_scores=True)
print("✅ Toxicity predictions done")

# Layer 4: Stance
stance_preds, stance_probs = stance_clf.predict(texts, return_probabilities=True)
print("✅ Stance predictions done")

# Layer 5: Intent
intent_preds, intent_probs = intent_clf.predict(texts, return_probabilities=True)
print("✅ Intent predictions done")

print("\n🎉 All layers applied successfully!")

## 12. Create Comprehensive Output DataFrame

In [ ]:
# Prepare predictions dictionary
predictions = {
    'emotion': emotion_preds,
    'emotion_probs': emotion_probs,
    'aspect': aspect_preds,
    'aspect_probs': aspect_probs,
    'toxicity': toxicity_labels,
    'toxicity_scores': toxicity_scores,
    'stance': stance_preds,
    'stance_probs': stance_probs,
    'intent': intent_preds,
    'intent_probs': intent_probs
}

# Create comprehensive dataframe
output_df = create_layered_output_dataframe(
    texts=texts,
    sentiment_labels=df['sentiment_auto'].values,
    sentiment_probs=df['sentiment_score'].values,
    predictions=predictions,
    low_conf_threshold=layered_config['low_confidence_threshold'],
    toxicity_threshold=layered_config['toxicity_threshold']
)

print("📊 Comprehensive Output DataFrame:")
print(f"Shape: {output_df.shape}")
print(f"\nColumns: {output_df.columns.tolist()}")

# Display output
output_df

## 13. Detailed View: Sample Predictions

In [ ]:
# Show detailed predictions for first 3 comments
print("🔍 Detailed Predictions:\n")
print("="*100)

for idx in range(min(3, len(output_df))):
    row = output_df.iloc[idx]
    print(f"\n📝 Comment #{idx+1}:")
    print(f"Text: {row['text'][:80]}..." if len(row['text']) > 80 else f"Text: {row['text']}")
    print(f"\n🎭 Sentiment: {row['sentiment']} (confidence: {row['sentiment_confidence']:.3f})")
    print(f"😊 Emotions: {row['emotions']}")
    print(f"🎯 Aspects: {row['aspects']}")
    print(f"⚠️ Toxicity: {row['toxicity_label']} (score: {row['toxicity_score']:.3f})")
    print(f"🔀 Stance: {row['stance']} (confidence: {row['stance_confidence']:.3f})")
    print(f"💬 Intent: {row['intent']} (confidence: {row['intent_confidence']:.3f})")
    print(f"🚩 Low Confidence? {row['low_confidence_flag']}")
    print("="*100)

## 14. Save Predictions to CSV

In [ ]:
# Save to CSV
output_path = '../data/processed/layered_predictions_real.csv'
output_df.to_csv(output_path, index=False, encoding='utf-8')

print(f"✅ Predictions saved to: {output_path}")
print(f"📊 Total predictions: {len(output_df)}")

## 15. Visualizations

In [ ]:
# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('🎯 Multi-Layer Analytics - Real Data Results', fontsize=16, fontweight='bold')

# 1. Sentiment distribution
sentiment_counts = output_df['sentiment'].value_counts()
axes[0, 0].bar(sentiment_counts.index, sentiment_counts.values, color=['green', 'gray', 'red'])
axes[0, 0].set_title('🎭 Sentiment Distribution')
axes[0, 0].set_xlabel('Sentiment')
axes[0, 0].set_ylabel('Count')

# 2. Emotion distribution (split multi-label)
emotion_split = output_df['emotions'].str.split(',').explode().str.strip()
emotion_counts = emotion_split.value_counts()
axes[0, 1].barh(emotion_counts.index, emotion_counts.values)
axes[0, 1].set_title('😊 Emotion Distribution')
axes[0, 1].set_xlabel('Count')

# 3. Aspect distribution (split multi-label)
aspect_split = output_df['aspects'].str.split(',').explode().str.strip()
aspect_counts = aspect_split.value_counts()
axes[0, 2].barh(aspect_counts.index, aspect_counts.values, color='orange')
axes[0, 2].set_title('🎯 Aspect Distribution')
axes[0, 2].set_xlabel('Count')

# 4. Toxicity distribution
toxicity_counts = output_df['toxicity_label'].value_counts()
colors_tox = ['red' if x == 'toxic' else 'green' for x in toxicity_counts.index]
axes[1, 0].pie(toxicity_counts.values, labels=toxicity_counts.index, autopct='%1.1f%%', colors=colors_tox)
axes[1, 0].set_title('⚠️ Toxicity Distribution')

# 5. Stance distribution
stance_counts = output_df['stance'].value_counts()
axes[1, 1].bar(stance_counts.index, stance_counts.values, color=['blue', 'red', 'gray'])
axes[1, 1].set_title('🔀 Stance Distribution')
axes[1, 1].set_xlabel('Stance')
axes[1, 1].set_ylabel('Count')
axes[1, 1].tick_params(axis='x', rotation=45)

# 6. Intent distribution
intent_counts = output_df['intent'].value_counts()
axes[1, 2].barh(intent_counts.index, intent_counts.values, color='purple')
axes[1, 2].set_title('💬 Intent Distribution')
axes[1, 2].set_xlabel('Count')

plt.tight_layout()
plt.savefig('../results/visualizations/layered_analytics_real_data.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualization saved to: ../results/visualizations/layered_analytics_real_data.png")

## 16. Generate Comprehensive Report

In [ ]:
# Generate comprehensive text report
report = create_comprehensive_report(
    output_df=output_df,
    original_df=df,
    include_temporal=False,  # No timestamp in dataset
    top_n=5
)

# Save report
report_path = '../results/reports/layered_analytics_real_data_report.txt'
os.makedirs(os.path.dirname(report_path), exist_ok=True)
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(report)

print("📄 Comprehensive Report:")
print("="*100)
print(report)
print("="*100)
print(f"\n✅ Report saved to: {report_path}")

## 17. Summary & Next Steps

In [ ]:
print("🎉 " + "="*100)
print("   MULTI-LAYER ANALYTICS COMPLETED!")
print("="*100)

print("\n✅ What was done:")
print("  1. ✅ Loaded real data (7 comments)")
print("  2. ✅ Generated dummy labels for demo")
print("  3. ✅ Trained 5 layer classifiers:")
print("     - Emotion (multi-label, 6 classes)")
print("     - Aspect (multi-label, 8 classes)")
print("     - Toxicity (binary)")
print("     - Stance (3-class)")
print("     - Intent (6-class)")
print("  4. ✅ Applied all layers on real data")
print("  5. ✅ Generated comprehensive output CSV")
print("  6. ✅ Created visualizations")
print("  7. ✅ Generated comprehensive report")

print("\n📦 Deliverables:")
print("  - Models: ../models/*.pkl (5 classifiers)")
print("  - Predictions: ../data/processed/layered_predictions_real.csv")
print("  - Visualization: ../results/visualizations/layered_analytics_real_data.png")
print("  - Report: ../results/reports/layered_analytics_real_data_report.txt")

print("\n⚠️ IMPORTANT - Next Steps for Production:")
print("  1. 📝 Label more data (300-500 samples per class)")
print("     - Follow: data/labelled/LABELING_GUIDE.md")
print("     - Use template: data/labelled/labelled_comments_template.csv")

print("\n  2. 🔄 Retrain with labeled data:")
print("     - Load labelled_comments.csv")
print("     - Split 80/20 train/test")
print("     - Train all layers")
print("     - Validate on test set")

print("\n  3. 🚀 Deploy:")
print("     - Integrate dengan existing sentiment pipeline")
print("     - Create Streamlit dashboard")
print("     - Setup real-time monitoring")

print("\n  4. 📊 Analyze & Iterate:")
print("     - Collect feedback")
print("     - Tune thresholds")
print("     - Expand training data")

print("\n💡 Quick commands:")
print("  - Load saved model: clf = EmotionClassifier.load('path/to/model.pkl')")
print("  - Predict new text: labels, probs = clf.predict(['new comment'])")
print("  - Update config: Edit config.yaml then reload")

print("\n🎓 Documentation:")
print("  - Quick Start: ../QUICKSTART_LAYERED.md")
print("  - Full Docs: ../README.md")
print("  - Project Summary: ../SUMMARY_LAYERED.md")

print("\n" + "="*100)